# __QM/MM protocol automation__

In [1]:
import os
import subprocess

# macOS
bash = '/opt/homebrew/bin/bash'
# linux
#bash = '/bin/bash'

def execute(command_str):
    subprocess.run(command_str, shell=True, executable=bash)

# execute('echo "sergi!"')

## __Manual steps__

In [2]:
# QM/MM simulation
# required 1: 
#   - 1_frames/frame_extraction/
#           - md_run, frames.ipynb, frames      # aquesta es la unica part manual
#   - src
#           - source code de la simulacio, si el lligand canvia, els inputs tleap i altres scripts tambe

In [3]:
# 
#   QM/MM SETUP OPTIONS
#
H = 10
dist_criterium = 10.0
dih_criterium = 60.0
lig_ID = '5-HETE'
num_precat = 1


# set up correct atoms for the setup depending on ligand and H number
if lig_ID == '5-HETE':
    if H == 10:
        gen_params_atom = 10581
        H_atom = 10583
        C_atom = 10581
        O_OH1_atom = 10552

    elif H == 13:
        gen_params_atom = 10574
        H_atom = 10576
        C_atom = 10574
        O_OH1_atom = 10552
    else:
        raise ValueError(f'H{H} is not H10 or H13!')
    
elif lig_ID == '5-HpETE':
    if H == 10:
        gen_params_atom = 10581
        H_atom = 10583
        C_atom = 10581
        O_OH1_atom = 10552
    elif H == 13:
        gen_params_atom = 10574
        H_atom = 10576
        C_atom = 10574
        O_OH1_atom = 10552
    else:
        raise ValueError(f'H{H} is not H10 or H13!')
else:
    raise ValueError(f'failed to identify ligand name {lig_ID} as 5-HETE or 5-HpETE.')

In [4]:
#
#   PATH SETUP
#
# FRAMES
frame_extraction = '../1_frames/frame_extraction'
frame_path = os.path.relpath(f'../1_frames/frame_extraction/frames_H{H}')
frame_preprocessing = '../1_frames/frame_preprocessing'
frame_source = '../1_frames/frame_source'

# CALCULATIONS
calculations = '../2_calculations'


# ligand specific documents
qm_list_name = f'qm_list_{lig_ID[2:]}'
tleap_name = f'tleap_{lig_ID[2:]}'

In [5]:
#
#   PREPROCESS THE MD TRAJECTORY AND EXTRACT PRECATALYTIC STRUCTURES
#

# copy extract_precatalytic.py to frame_extraction and execture the script 
execute(f'cp extract_precatalytic.py {frame_extraction}')
execute(f'cd {frame_extraction}; python extract_precatalytic.py -m {lig_ID} -H {H} -d {dist_criterium} -l {dih_criterium} -n {num_precat} 2>&1 | tee extract_H{H}.txt')

/Users/sergiortizropero/miniconda3/envs/md_traj/lib/python3.12/site-packages/Bio/Application/__init__.py:39: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(
EMDA version:	1.0.0a5
loading prmtop	/Users/sergiortizropero/TFG_chem/ML-MM_setup/example/1_frames/frame_extraction/md_run/h15LOX_5-HETE_343_12_mod_norelaxed_4amber_solv.prmtop
['/Users/sergiortizropero/TFG_chem/ML-MM_setup/example/1_frames/frame_extraction/md_run/prod/out/prod_1.nc']
/Users/sergiortizropero/miniconda3/envs/md_traj/lib/python3.12/site-packages/MDAnalysis/topology/TOPParser.py:342: UserWarning: Unknown ATOMIC_NUMBER value found for some atoms, these have been given an empty eleme

In [ ]:
#
#   GENERATE PARAMETERS FOR EACH PRECATALYTIC STRUCTURE
#

# select frames_int random frames
selected_list = [int(x.strip('.pdb')) for x in os.listdir(frame_path) if ('.pdb' in x) and ('_' not in x)]

# create frame_preprocessing and copy src code
if not os.path.exists(frame_preprocessing):
    os.mkdir(frame_preprocessing)

# copy necessary preprocessing scripts and tleap folder with params. 
execute(f'cp -r {tleap_name} {os.path.join(frame_preprocessing, 'tleap')}')
execute(f'cp amber_to_chemshell_modeller.py {frame_preprocessing}')
execute(f'cp generate_params.sh {frame_preprocessing}')
execute(f'cd {frame_preprocessing}; sed "s/<CATOM>/{C_atom}/g" generate_params.sh > generate_params_mod.sh; rm generate_params.sh; mv generate_params_mod.sh generate_params.sh')

# copy frames to their respective folders
for frame in selected_list:
    execute(f'cp {os.path.join(frame_path, str(frame)+'.pdb')} {frame_preprocessing}')

# process the frames with tleap via `generate_params.sh` and `amber_to_chemshell_modeller.py`
execute(f'cd {frame_preprocessing}; bash generate_params.sh')

/Users/sergiortizropero/TFG_chem/ML-MM_setup/example/1_frames/frame_preprocessing
12.pdb
12
12_solv.prmtop
-I: Adding /Users/sergiortizropero/Amber/amber24/dat/leap/prep to search path.
-I: Adding /Users/sergiortizropero/Amber/amber24/dat/leap/lib to search path.
-I: Adding /Users/sergiortizropero/Amber/amber24/dat/leap/parm to search path.
-I: Adding /Users/sergiortizropero/Amber/amber24/dat/leap/cmd to search path.
-f: Source tleap.inp.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./tleap.inp
----- Source: /Users/sergiortizropero/Amber/amber24/dat/leap/cmd/leaprc.protein.ff19SB
----- Source of /Users/sergiortizropero/Amber/amber24/dat/leap/cmd/leaprc.protein.ff19SB done
Log file: ./leap.log
Loading parameters: /Users/sergiortizropero/Amber/amber24/dat/leap/parm/parm19.dat
Reading title:
PARM99 + frcmod.ff99SB + frcmod.parmbsc0 + OL3 for RNA + ff19SB
Loading parameters: /Users/sergiortizropero/Amber/amber24/dat/leap/parm/frcmod.ff19SB
Reading force field modification type fi

/Users/sergiortizropero/miniconda3/envs/md_traj/lib/python3.12/site-packages/MDAnalysis/core/universe.py:150: UserWarning: No coordinate reader found for 12_solv.prmtop. Skipping this file.
  warnings.warn('No coordinate reader found for {}. Skipping '


Cropped topology and coordinates have been saved as '12_solv_cropped.prmtop', '12_solv_cropped.inpcrd' and '12_solv_cropped.pdb'
New parameters and coordinates files of the cropped system will be used in future actions.
2106 atoms have been set as active for the QM/MM calculation using ChemShell.
8 atom types from MCPB have been changed. 
prmtop fixed!


/Users/sergiortizropero/miniconda3/envs/md_traj/lib/python3.12/site-packages/MDAnalysis/core/universe.py:150: UserWarning: No coordinate reader found for 12_solv_cropped.prmtop. Skipping this file.
  warnings.warn('No coordinate reader found for {}. Skipping '


In [7]:
# add relevant parameters as src for each frame 
if not os.path.exists(frame_source):
    os.mkdir(frame_source)

for frame in selected_list:
    if not os.path.exists(os.path.join(frame_source, str(frame))):
        os.mkdir(os.path.join(frame_source, str(frame)))
    
    execute(f'cd {os.path.join(frame_preprocessing, str(frame), 'tleap')}; cp {frame}_solv_cropped_mod.prmtop ../../../frame_source/{frame}')
    execute(f'cd {os.path.join(frame_preprocessing, str(frame), 'tleap')}; cp {frame}_solv_cropped.inpcrd ../../../frame_source/{frame}')
    execute(f'cd {os.path.join(frame_preprocessing, str(frame), 'tleap')}; cp {frame}_solv_cropped.pdb ../../../frame_source/{frame}')
    execute(f'cd {os.path.join(frame_preprocessing, str(frame), 'tleap')}; cp {frame}_solv.act_list ../../../frame_source/{frame}')

In [8]:


# prepare the QM/MM setup for each frame
for frame in selected_list:
    if not os.path.exists(os.path.join(calculations, str(frame))):
        
        # 1_qm_region
        os.makedirs(os.path.join(calculations, str(frame)))
        os.mkdir(os.path.join(calculations, str(frame), '1_qm_region'))
        execute(f'cp {qm_list_name} {os.path.join(calculations, str(frame), '1_qm_region', 'qm_list')}')
        execute(f'cp {os.path.join(frame_source, str(frame), f'{frame}_solv_cropped.pdb')} {os.path.join(calculations, str(frame), '1_qm_region')}')
        execute(f'cp atoms_list_to_pdb.py {os.path.join(calculations, str(frame), '1_qm_region')}')
        print('converting qm_list to qm_list.pdb')
        execute(f'cd {os.path.join(calculations, str(frame), '1_qm_region')}; python atoms_list_to_pdb.py \'qm_list\' {frame}_solv_cropped.pdb qm_list')

        # open chimera with the #0 qm_list.pdb and #1 the protein
        # run the script
        execute(f'cp qm_region_view.py {os.path.join(calculations, str(frame), '1_qm_region')}')
        execute(f'cd {os.path.join(calculations, str(frame), '1_qm_region')}; sed "s:<PATH>:{os.path.abspath(os.path.join(calculations, str(frame), '1_qm_region', 'view.py'))}:g" qm_region_view.py > qm_region_view_mod.py; rm qm_region_view.py; mv qm_region_view_mod.py qm_region_view.py')

        # 2_opt
        os.mkdir(os.path.join(calculations, str(frame), '2_opt'))
        execute(f'cp {os.path.join(frame_source, str(frame), '*')} {os.path.join(calculations, str(frame), '2_opt')}')
        execute(f'cp {qm_list_name} {os.path.join(calculations, str(frame), '2_opt', 'qm_list')}')
        execute(f'cp opt_job.sh {os.path.join(calculations, str(frame), '2_opt')}')
        execute(f'cp opt_plot.py {os.path.join(calculations, str(frame), '2_opt')}')
        execute(f'cp opt.chm {os.path.join(calculations, str(frame), '2_opt')}')
        execute(f'cd {os.path.join(calculations, str(frame), '2_opt')}; sed "s/<FRAME>/{frame}/g" opt_job.sh > opt_job_mod.sh; rm opt_job.sh; mv opt_job_mod.sh opt_job.sh')
        execute(f'cd {os.path.join(calculations, str(frame), '2_opt')}; sed "s/<FRAME>/{frame}/g" opt.chm > opt_mod.chm; rm opt.chm; mv opt_mod.chm opt.chm')

        # 3_scan
        os.mkdir(os.path.join(calculations, str(frame), '3_scan'))
        execute(f'cp {os.path.join(os.path.join(calculations, str(frame), '2_opt'), f'{frame}_solv_cropped_mod.prmtop')} {os.path.join(calculations, str(frame), '3_scan')}')
        execute(f'cp {os.path.join(os.path.join(calculations, str(frame), '2_opt'), f'{frame}_solv_cropped.inpcrd')} {os.path.join(calculations, str(frame), '3_scan')}')
        execute(f'cp {os.path.join(os.path.join(calculations, str(frame), '2_opt'), f'{frame}_solv.act_list')} {os.path.join(calculations, str(frame), '3_scan')}')
        execute(f'cp {os.path.join(os.path.join(calculations, str(frame), '2_opt'), f'qm_list')} {os.path.join(calculations, str(frame), '3_scan')}')
        execute(f'cp scan_job.sh {os.path.join(calculations, str(frame), '3_scan')}')
        execute(f'cp scan_plot.py {os.path.join(calculations, str(frame), '3_scan')}')
        execute(f'cp scan.chm {os.path.join(calculations, str(frame), '3_scan')}')
        execute(f'cd {os.path.join(calculations, str(frame), '3_scan')}; sed "s/<FRAME>/{frame}/g" scan_job.sh > scan_job_mod.sh; rm scan_job.sh; mv scan_job_mod.sh scan_job.sh')
        execute(f'cd {os.path.join(calculations, str(frame), '3_scan')}; sed "s/<FRAME>/{frame}/g" scan.chm > scan_mod.chm; rm scan.chm; mv scan_mod.chm scan.chm')
        execute(f'cd {os.path.join(calculations, str(frame), '3_scan')}; sed "s/<HYDROGEN>/{H}/g" scan.chm > scan_mod.chm; rm scan.chm; mv scan_mod.chm scan.chm')
        execute(f'cd {os.path.join(calculations, str(frame), '3_scan')}; sed "s/<OHATOM>/{O_OH1_atom}/g" scan.chm > scan_mod.chm; rm scan.chm; mv scan_mod.chm scan.chm')
        execute(f'cd {os.path.join(calculations, str(frame), '3_scan')}; sed "s/<CXATOM>/{C_atom}/g" scan.chm > scan_mod.chm; rm scan.chm; mv scan_mod.chm scan.chm')
        execute(f'cd {os.path.join(calculations, str(frame), '3_scan')}; sed "s/<HXATOM>/{H_atom}/g" scan.chm > scan_mod.chm; rm scan.chm; mv scan_mod.chm scan.chm')
        execute(f'cd {os.path.join(calculations, str(frame), '3_scan')}; sed "s/<FRAME>/{frame}/g" scan_plot.py > scan_plot_mod.py; rm scan_plot.py; mv scan_plot_mod.py scan_plot.py')
        os.mkdir(os.path.join(calculations, str(frame), '3_scan', 'structures'))
        execute(f'cp multiframe.py {os.path.join(calculations, str(frame), '3_scan', 'structures')}')
        execute(f'cp scan_view.py {os.path.join(calculations, str(frame), '3_scan', 'structures')}')
        execute(f'cd {os.path.join(calculations, str(frame), '3_scan', 'structures')}; sed "s:<PATH>:{os.path.abspath(os.path.join(calculations, str(frame), '3_scan', 'structures', 'multiframe_view.py'))}:g" scan_view.py > scan_view_mod.py; rm scan_view.py; mv scan_view_mod.py scan_view.py')

        # 4_TS_opt
        os.mkdir(os.path.join(calculations, str(frame), '4_TS_opt'))
        execute(f'cp {os.path.join(frame_source, str(frame), '*.prmtop')} {os.path.join(calculations, str(frame), '4_TS_opt')}')
        execute(f'cp {os.path.join(frame_source, str(frame), '*.inpcrd')} {os.path.join(calculations, str(frame), '4_TS_opt')}')
        execute(f'cp {os.path.join(frame_source, str(frame), '*.act_list')} {os.path.join(calculations, str(frame), '4_TS_opt')}')
        execute(f'cp {qm_list_name} {os.path.join(calculations, str(frame), '4_TS_opt', 'qm_list')}')
        execute(f'cp ts_info.txt {os.path.join(calculations, str(frame), '4_TS_opt')}')
        execute(f'cp ts_job.sh {os.path.join(calculations, str(frame), '4_TS_opt')}')
        execute(f'cp ts.chm {os.path.join(calculations, str(frame), '4_TS_opt')}')
        execute(f'cp opt_plot.py {os.path.join(calculations, str(frame), '4_TS_opt')}')
        execute(f'cd {os.path.join(calculations, str(frame), '4_TS_opt')}; mv opt_plot.py ts_plot.py')

converting qm_list to qm_list.pdb
The zone has 75 atoms (if it is a QM zone, link atoms are not counted)


/Users/sergiortizropero/miniconda3/envs/md_traj/lib/python3.12/site-packages/MDAnalysis/coordinates/PDB.py:777: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/Users/sergiortizropero/miniconda3/envs/md_traj/lib/python3.12/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"
/Users/sergiortizropero/miniconda3/envs/md_traj/lib/python3.12/site-packages/MDAnalysis/coordinates/PDB.py:1201: UserWarning: Found missing chainIDs. Corresponding atoms will use value of 'X'
  warnings.warn("Found missing chainIDs."


In [9]:
# mkdir 2_calculations
# mkdir <frame>
# mkdir 1_qm_region, 2_opt, 3_scan

# 1_qm_region
# 

# cp <FEAME>_cropped.pdb, qm_list, atoms_list_to_pdb.py
# cp visualization.py (for chimera, open cropped.pdb and qm_list.pdb IN THAT ORDER, then execute visualization.py to view the qm_region in chimera)
# CHIMERA SHOULD SAVE THE SESSION ITSELF AND GENERATE THE PY AUTOMATICALLY

# 2_opt
# cp <FRAME>_cropped_mod.prmtop, FRAME>_cropped.inpcrd <FRAME>_cropped.pdb <FRAME>_cropped.act_list qm_list job.sh MODIFICAT i opt.chm MODIFICAT
# MANUAL: copiar a picard i enviar cada treball (si son molts potser cal una script, pero meh)

# 3_scan
# todo

In [10]:
# MANUAL STEPS NEEDEED:
# ONCE THE OPT HAS FINISHED, COPY THE OUTPUT PDB AND C TO SCAN DIRECTORY


# TO SUBMIT THE SCAN JOB, CHANGE THE SCANDIR of scan_job.sh, to that of picard. 